# Deep Convolution-GAN (DC-GAN)

## 知识点: 

#### 1. Fully-Convolutional Neural Network (Transposed convolution)
#### 2. PyTorch Implementation pipeline 
#### 3. How to optimize a minimax saddle point problem

In [1]:
import argparse #引入超参数设置
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim  #引入PyTorch中的优化模块
import torch.utils.data
import torchvision.datasets as dset  #提供数据库，可以进行数据预处理
import torchvision.transforms as transforms
import torchvision.utils as vutils
import test_data

## 设置程序的超参数，包括CNN中的channels个数，同时也支持断点续训练

In [2]:
parser = argparse.ArgumentParser()   #每个超参数给一个初始值（推荐的写法）
parser.add_argument('--dataset', type=str, default="mnist")
parser.add_argument('--dataroot', type=str, default="./data")
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64, help="number of frames/channels in the generator")
parser.add_argument('--ndf', type=int, default=64, help="number of frames/channels in the discriminator")
parser.add_argument('--niter', type=int, default=30, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', type=bool, default=True, help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='./DCGAN_output_MNIST_512_again', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, default=42, help='manual seed')

_StoreAction(option_strings=['--manualSeed'], dest='manualSeed', nargs=None, const=None, default=42, type=<class 'int'>, choices=None, help='manual seed', metavar=None)

In [3]:
# opt = parser.parse_args()
opt = parser.parse_args(args=[])
print(opt)

try:
    os.makedirs(opt.outf)
except OSError:
    pass
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
cudnn.benchmark = True

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

Namespace(batchSize=64, beta1=0.5, cuda=True, dataroot='./data', dataset='mnist', imageSize=64, lr=0.0002, manualSeed=42, ndf=64, netD='', netG='', ngf=64, ngpu=1, niter=30, nz=100, outf='./DCGAN_output_MNIST_512_again', workers=2)
Random Seed:  42


### 载入MNIST数据集并对数据集进行预处理。注意：对于非线性神经网络来说，数据预处理对模型的实际效果影响会特别大，因此切记进行数据预处理。

In [4]:

dataset = dset.MNIST(root=opt.dataroot, download=True,
                   transform=transforms.Compose([
                       transforms.Resize(opt.imageSize),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,), (0.5,)),
                   ]))

'''
dataSet=test_data.Flower('C:/Users/HP/Desktop/train/oxford102/',transform=transforms.Compose([
    transforms.Resize(opt.imageSize), #缩放图片，最短边的长为opt.imageSize像素,
    transforms.CenterCrop(opt.imageSize), #从中间切出opt.imageSize * opt.imageSize的图片
    transforms.ToTensor(), 
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]) 
]))
dataloader = torch.utils.data.DataLoader(dataSet, batch_size=opt.batchSize, shuffle=True)
'''
nc=1

In [5]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))

device = torch.device("cuda:0" if opt.cuda else "cpu")
ngpu = int(opt.ngpu)
nz = int(opt.nz)
ngf = int(opt.ngf)
ndf = int(opt.ndf)

### 神经网络权重的初始化，只需要对卷积层和BatchNorm进行特殊处理，全连接部分直接使用Xavier 初始化方式即可

In [6]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

## 图片生成器，Fully Convolutional Neural Network

1. 和Keras/Tensorflow一样有方便的nn.Sequential()接口用来定义一个序列模型，整个代码就像搭积木一样简单
2. 模型的输入是纯粹的高斯噪声，模型的输出就是 nc x 64 x 64维的图片
3. 图片的维度越来越大

In [7]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. 512 x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. 256 x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. 128 x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. 64 x 32 x 32
            nn.ConvTranspose2d( ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. 3 x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [8]:
netG = Generator(ngpu).to(device)
netG.apply(weights_init)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG))
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


## 图片判别器：基于CNN的图像分类

1. 传统的CNN，注意我们这里为了训练效果更好使用了LeakyReLU而不是简单的ReLU
2. 图片的维度越来越小，最后一层就是全连分类

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

In [ ]:
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))
print(netD)


Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


## 定义损失函数: $\ell(\hat{y}, y) = y\log \hat{y} + (1-y)\log(1 - \hat{y})$

In [ ]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(opt.batchSize, nz, 1, 1, device=device)
real_label = 1
fake_label = 0


## 使用Adam 优化方法来对目标函数进行优化


### 我们这里有两个模型，一个D：图片判别器，一个G：图片生成器

In [ ]:
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))

In [ ]:
d_losses = np.zeros(opt.niter)
g_losses = np.zeros(opt.niter)
for epoch in range(opt.niter):
    d_loss, g_loss = 0.0, 0.0
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()
        d_loss += errD.item()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        g_loss += errG.item()
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, opt.niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % opt.outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (opt.outf, epoch),
                    normalize=True)
    # Update both losses.
    g_losses[epoch] = g_loss
    d_losses[epoch] = d_loss
    

[0/30][0/938] Loss_D: 1.8170 Loss_G: 6.6004 D(x): 0.7519 D(G(z)): 0.7202 / 0.0022
[0/30][1/938] Loss_D: 0.3430 Loss_G: 6.8027 D(x): 0.9578 D(G(z)): 0.2237 / 0.0015
[0/30][2/938] Loss_D: 0.2629 Loss_G: 6.7060 D(x): 0.9636 D(G(z)): 0.1830 / 0.0016
[0/30][3/938] Loss_D: 0.1844 Loss_G: 6.9255 D(x): 0.9557 D(G(z)): 0.1186 / 0.0014
[0/30][4/938] Loss_D: 0.1616 Loss_G: 7.6241 D(x): 0.9802 D(G(z)): 0.1226 / 0.0008
[0/30][5/938] Loss_D: 0.1208 Loss_G: 7.5186 D(x): 0.9619 D(G(z)): 0.0723 / 0.0010
[0/30][6/938] Loss_D: 0.1222 Loss_G: 7.8620 D(x): 0.9748 D(G(z)): 0.0847 / 0.0006
[0/30][7/938] Loss_D: 0.1199 Loss_G: 8.4488 D(x): 0.9687 D(G(z)): 0.0750 / 0.0004
[0/30][8/938] Loss_D: 0.0532 Loss_G: 7.8735 D(x): 0.9813 D(G(z)): 0.0322 / 0.0006
[0/30][9/938] Loss_D: 0.0872 Loss_G: 9.0274 D(x): 0.9913 D(G(z)): 0.0705 / 0.0002
[0/30][10/938] Loss_D: 0.0440 Loss_G: 8.2480 D(x): 0.9801 D(G(z)): 0.0229 / 0.0004
[0/30][11/938] Loss_D: 0.1005 Loss_G: 9.1158 D(x): 0.9719 D(G(z)): 0.0590 / 0.0002
[0/30][12/938]

[0/30][195/938] Loss_D: 0.1283 Loss_G: 4.4220 D(x): 0.9542 D(G(z)): 0.0560 / 0.0159
[0/30][196/938] Loss_D: 0.1111 Loss_G: 3.7297 D(x): 0.9299 D(G(z)): 0.0285 / 0.0303
[0/30][197/938] Loss_D: 0.1007 Loss_G: 4.6571 D(x): 0.9752 D(G(z)): 0.0708 / 0.0125
[0/30][198/938] Loss_D: 0.0812 Loss_G: 4.5838 D(x): 0.9556 D(G(z)): 0.0249 / 0.0126
[0/30][199/938] Loss_D: 0.0916 Loss_G: 3.9958 D(x): 0.9516 D(G(z)): 0.0302 / 0.0271
[0/30][200/938] Loss_D: 0.0637 Loss_G: 4.2477 D(x): 0.9722 D(G(z)): 0.0300 / 0.0200
[0/30][201/938] Loss_D: 0.0536 Loss_G: 4.8537 D(x): 0.9831 D(G(z)): 0.0343 / 0.0124
[0/30][202/938] Loss_D: 0.0829 Loss_G: 4.6180 D(x): 0.9626 D(G(z)): 0.0410 / 0.0159
[0/30][203/938] Loss_D: 0.0477 Loss_G: 4.6993 D(x): 0.9726 D(G(z)): 0.0183 / 0.0129
[0/30][204/938] Loss_D: 0.0767 Loss_G: 3.8120 D(x): 0.9532 D(G(z)): 0.0220 / 0.0287
[0/30][205/938] Loss_D: 0.1786 Loss_G: 6.5866 D(x): 0.9789 D(G(z)): 0.1359 / 0.0020
[0/30][206/938] Loss_D: 0.2346 Loss_G: 2.6266 D(x): 0.8304 D(G(z)): 0.0050 /

[0/30][389/938] Loss_D: 0.4118 Loss_G: 4.7938 D(x): 0.9717 D(G(z)): 0.2929 / 0.0130
[0/30][390/938] Loss_D: 0.1596 Loss_G: 4.9409 D(x): 0.8903 D(G(z)): 0.0286 / 0.0134
[0/30][391/938] Loss_D: 0.1961 Loss_G: 3.5393 D(x): 0.8566 D(G(z)): 0.0180 / 0.0443
[0/30][392/938] Loss_D: 0.1707 Loss_G: 3.1225 D(x): 0.9580 D(G(z)): 0.1076 / 0.0629
[0/30][393/938] Loss_D: 0.1529 Loss_G: 4.0949 D(x): 0.9771 D(G(z)): 0.1129 / 0.0250
[0/30][394/938] Loss_D: 0.2107 Loss_G: 3.4793 D(x): 0.8938 D(G(z)): 0.0827 / 0.0457
[0/30][395/938] Loss_D: 0.1842 Loss_G: 3.3422 D(x): 0.9184 D(G(z)): 0.0840 / 0.0492
[0/30][396/938] Loss_D: 0.2120 Loss_G: 3.2712 D(x): 0.9101 D(G(z)): 0.0990 / 0.0527
[0/30][397/938] Loss_D: 0.2341 Loss_G: 3.4777 D(x): 0.9120 D(G(z)): 0.1183 / 0.0440
[0/30][398/938] Loss_D: 0.2202 Loss_G: 3.0824 D(x): 0.8712 D(G(z)): 0.0656 / 0.0683
[0/30][399/938] Loss_D: 0.3115 Loss_G: 3.7257 D(x): 0.9206 D(G(z)): 0.1745 / 0.0368
[0/30][400/938] Loss_D: 0.2913 Loss_G: 2.6058 D(x): 0.8238 D(G(z)): 0.0699 /

[0/30][583/938] Loss_D: 0.4985 Loss_G: 1.2743 D(x): 0.7006 D(G(z)): 0.0708 / 0.3399
[0/30][584/938] Loss_D: 0.6415 Loss_G: 4.5349 D(x): 0.9669 D(G(z)): 0.3915 / 0.0178
[0/30][585/938] Loss_D: 0.7940 Loss_G: 1.1564 D(x): 0.5179 D(G(z)): 0.0245 / 0.3927
[0/30][586/938] Loss_D: 0.5569 Loss_G: 4.2240 D(x): 0.9777 D(G(z)): 0.3793 / 0.0222
[0/30][587/938] Loss_D: 0.3505 Loss_G: 2.4470 D(x): 0.7780 D(G(z)): 0.0576 / 0.1215
[0/30][588/938] Loss_D: 0.2607 Loss_G: 2.2807 D(x): 0.8879 D(G(z)): 0.1180 / 0.1396
[0/30][589/938] Loss_D: 0.3852 Loss_G: 3.5897 D(x): 0.9188 D(G(z)): 0.2372 / 0.0372
[0/30][590/938] Loss_D: 0.3976 Loss_G: 1.8652 D(x): 0.7393 D(G(z)): 0.0643 / 0.2036
[0/30][591/938] Loss_D: 0.4412 Loss_G: 3.2231 D(x): 0.9144 D(G(z)): 0.2722 / 0.0497
[0/30][592/938] Loss_D: 0.4162 Loss_G: 1.8956 D(x): 0.7227 D(G(z)): 0.0493 / 0.1934
[0/30][593/938] Loss_D: 0.4823 Loss_G: 4.1313 D(x): 0.9391 D(G(z)): 0.3043 / 0.0262
[0/30][594/938] Loss_D: 0.5369 Loss_G: 1.1287 D(x): 0.6423 D(G(z)): 0.0314 /

[0/30][777/938] Loss_D: 0.2382 Loss_G: 2.6438 D(x): 0.8580 D(G(z)): 0.0704 / 0.0936
[0/30][778/938] Loss_D: 0.2634 Loss_G: 2.6400 D(x): 0.9065 D(G(z)): 0.1385 / 0.0875
[0/30][779/938] Loss_D: 0.3441 Loss_G: 2.4713 D(x): 0.8508 D(G(z)): 0.1450 / 0.1071
[0/30][780/938] Loss_D: 0.3305 Loss_G: 3.5579 D(x): 0.9336 D(G(z)): 0.2067 / 0.0383
[0/30][781/938] Loss_D: 0.3740 Loss_G: 2.0286 D(x): 0.7410 D(G(z)): 0.0375 / 0.1699
[0/30][782/938] Loss_D: 0.3211 Loss_G: 2.5506 D(x): 0.9178 D(G(z)): 0.1925 / 0.0982
[0/30][783/938] Loss_D: 0.2819 Loss_G: 2.6752 D(x): 0.8711 D(G(z)): 0.1206 / 0.0872
[0/30][784/938] Loss_D: 0.2587 Loss_G: 2.7143 D(x): 0.8785 D(G(z)): 0.1042 / 0.1007
[0/30][785/938] Loss_D: 0.3053 Loss_G: 2.8271 D(x): 0.8826 D(G(z)): 0.1401 / 0.0794
[0/30][786/938] Loss_D: 0.2267 Loss_G: 2.8727 D(x): 0.8856 D(G(z)): 0.0914 / 0.0779
[0/30][787/938] Loss_D: 0.2550 Loss_G: 3.0421 D(x): 0.9101 D(G(z)): 0.1357 / 0.0681
[0/30][788/938] Loss_D: 0.2638 Loss_G: 2.6578 D(x): 0.8686 D(G(z)): 0.0958 /

...

[26/30][696/938] Loss_D: 0.0158 Loss_G: 6.4953 D(x): 0.9990 D(G(z)): 0.0146 / 0.0023
[26/30][697/938] Loss_D: 0.0102 Loss_G: 6.4761 D(x): 0.9933 D(G(z)): 0.0034 / 0.0023
[26/30][698/938] Loss_D: 0.0245 Loss_G: 4.3838 D(x): 0.9782 D(G(z)): 0.0021 / 0.0192
[26/30][699/938] Loss_D: 0.0034 Loss_G: 5.4279 D(x): 0.9993 D(G(z)): 0.0027 / 0.0071
[26/30][700/938] Loss_D: 0.0141 Loss_G: 6.0750 D(x): 0.9992 D(G(z)): 0.0131 / 0.0033
[26/30][701/938] Loss_D: 0.0082 Loss_G: 6.6232 D(x): 0.9977 D(G(z)): 0.0058 / 0.0022
[26/30][702/938] Loss_D: 0.0119 Loss_G: 6.3133 D(x): 0.9924 D(G(z)): 0.0041 / 0.0036
[26/30][703/938] Loss_D: 0.0060 Loss_G: 6.4837 D(x): 0.9962 D(G(z)): 0.0022 / 0.0025
[26/30][704/938] Loss_D: 0.0077 Loss_G: 6.0920 D(x): 0.9988 D(G(z)): 0.0064 / 0.0035
[26/30][705/938] Loss_D: 0.0074 Loss_G: 6.4214 D(x): 0.9964 D(G(z)): 0.0037 / 0.0027
[26/30][706/938] Loss_D: 0.0085 Loss_G: 5.9665 D(x): 0.9956 D(G(z)): 0.0041 / 0.0037
[26/30][707/938] Loss_D: 0.0089 Loss_G: 6.1030 D(x): 0.9941 D(G(z

[26/30][888/938] Loss_D: 1.0788 Loss_G: 4.5022 D(x): 0.9453 D(G(z)): 0.5622 / 0.0198
[26/30][889/938] Loss_D: 1.8153 Loss_G: 1.0109 D(x): 0.3073 D(G(z)): 0.0878 / 0.4635
[26/30][890/938] Loss_D: 1.1134 Loss_G: 3.3300 D(x): 0.9242 D(G(z)): 0.5595 / 0.0636
[26/30][891/938] Loss_D: 0.7707 Loss_G: 2.2228 D(x): 0.6049 D(G(z)): 0.1265 / 0.1649
[26/30][892/938] Loss_D: 0.7085 Loss_G: 3.1592 D(x): 0.8222 D(G(z)): 0.3322 / 0.0643
[26/30][893/938] Loss_D: 0.8323 Loss_G: 2.7198 D(x): 0.7023 D(G(z)): 0.2912 / 0.1007
[26/30][894/938] Loss_D: 0.6656 Loss_G: 3.0444 D(x): 0.7948 D(G(z)): 0.2435 / 0.0771
[26/30][895/938] Loss_D: 1.0668 Loss_G: 1.7293 D(x): 0.6127 D(G(z)): 0.3219 / 0.2199
[26/30][896/938] Loss_D: 0.7352 Loss_G: 5.0650 D(x): 0.8706 D(G(z)): 0.3854 / 0.0091
[26/30][897/938] Loss_D: 1.3835 Loss_G: 0.4896 D(x): 0.3580 D(G(z)): 0.0665 / 0.6795
[26/30][898/938] Loss_D: 2.0869 Loss_G: 6.4206 D(x): 0.9564 D(G(z)): 0.7839 / 0.0044
[26/30][899/938] Loss_D: 1.9937 Loss_G: 2.9491 D(x): 0.2443 D(G(z

[27/30][144/938] Loss_D: 0.0063 Loss_G: 6.2747 D(x): 0.9973 D(G(z)): 0.0035 / 0.0029
[27/30][145/938] Loss_D: 0.0117 Loss_G: 5.7904 D(x): 0.9976 D(G(z)): 0.0092 / 0.0050
[27/30][146/938] Loss_D: 0.0264 Loss_G: 4.6703 D(x): 0.9810 D(G(z)): 0.0070 / 0.0140
[27/30][147/938] Loss_D: 0.0055 Loss_G: 5.7446 D(x): 0.9982 D(G(z)): 0.0038 / 0.0046
[27/30][148/938] Loss_D: 0.0240 Loss_G: 5.1121 D(x): 0.9918 D(G(z)): 0.0155 / 0.0089
[27/30][149/938] Loss_D: 0.0127 Loss_G: 5.9377 D(x): 0.9972 D(G(z)): 0.0097 / 0.0042
[27/30][150/938] Loss_D: 0.0079 Loss_G: 6.4460 D(x): 0.9962 D(G(z)): 0.0041 / 0.0030
[27/30][151/938] Loss_D: 0.0167 Loss_G: 5.3717 D(x): 0.9907 D(G(z)): 0.0073 / 0.0077
[27/30][152/938] Loss_D: 0.0096 Loss_G: 5.9071 D(x): 0.9942 D(G(z)): 0.0037 / 0.0045
[27/30][153/938] Loss_D: 0.0069 Loss_G: 6.1842 D(x): 0.9967 D(G(z)): 0.0035 / 0.0038
[27/30][154/938] Loss_D: 0.0161 Loss_G: 5.8673 D(x): 0.9959 D(G(z)): 0.0116 / 0.0056
[27/30][155/938] Loss_D: 0.0100 Loss_G: 6.1238 D(x): 0.9951 D(G(z

[27/30][336/938] Loss_D: 0.0113 Loss_G: 6.4111 D(x): 0.9981 D(G(z)): 0.0092 / 0.0031
[27/30][337/938] Loss_D: 0.0034 Loss_G: 7.6152 D(x): 0.9979 D(G(z)): 0.0013 / 0.0010
[27/30][338/938] Loss_D: 0.0099 Loss_G: 6.2905 D(x): 0.9920 D(G(z)): 0.0018 / 0.0036
[27/30][339/938] Loss_D: 0.0035 Loss_G: 6.5951 D(x): 0.9980 D(G(z)): 0.0015 / 0.0022
[27/30][340/938] Loss_D: 0.0022 Loss_G: 7.2627 D(x): 0.9987 D(G(z)): 0.0009 / 0.0011
[27/30][341/938] Loss_D: 0.0046 Loss_G: 6.4259 D(x): 0.9990 D(G(z)): 0.0037 / 0.0027
[27/30][342/938] Loss_D: 0.0061 Loss_G: 6.8590 D(x): 0.9990 D(G(z)): 0.0050 / 0.0022
[27/30][343/938] Loss_D: 0.0067 Loss_G: 6.5910 D(x): 0.9986 D(G(z)): 0.0053 / 0.0022
[27/30][344/938] Loss_D: 0.0053 Loss_G: 7.2691 D(x): 0.9959 D(G(z)): 0.0011 / 0.0011
[27/30][345/938] Loss_D: 0.0040 Loss_G: 6.9327 D(x): 0.9973 D(G(z)): 0.0013 / 0.0015
[27/30][346/938] Loss_D: 0.0034 Loss_G: 7.2178 D(x): 0.9979 D(G(z)): 0.0012 / 0.0015
[27/30][347/938] Loss_D: 0.0066 Loss_G: 6.1539 D(x): 0.9963 D(G(z

[27/30][528/938] Loss_D: 0.8067 Loss_G: 2.2403 D(x): 0.8265 D(G(z)): 0.4066 / 0.1424
[27/30][529/938] Loss_D: 0.5366 Loss_G: 2.3882 D(x): 0.7628 D(G(z)): 0.1812 / 0.1357
[27/30][530/938] Loss_D: 0.7585 Loss_G: 1.2512 D(x): 0.6292 D(G(z)): 0.1789 / 0.3375
[27/30][531/938] Loss_D: 0.7866 Loss_G: 1.6324 D(x): 0.7654 D(G(z)): 0.3542 / 0.2399
[27/30][532/938] Loss_D: 0.7021 Loss_G: 2.3106 D(x): 0.7949 D(G(z)): 0.3201 / 0.1281
[27/30][533/938] Loss_D: 0.5829 Loss_G: 2.3441 D(x): 0.7576 D(G(z)): 0.2078 / 0.1441
[27/30][534/938] Loss_D: 0.6440 Loss_G: 1.5296 D(x): 0.6708 D(G(z)): 0.1423 / 0.2810
[27/30][535/938] Loss_D: 0.7012 Loss_G: 2.2764 D(x): 0.8464 D(G(z)): 0.3632 / 0.1418
[27/30][536/938] Loss_D: 0.5927 Loss_G: 2.0455 D(x): 0.7287 D(G(z)): 0.1946 / 0.1653
[27/30][537/938] Loss_D: 0.6227 Loss_G: 2.2026 D(x): 0.7799 D(G(z)): 0.2509 / 0.1676
[27/30][538/938] Loss_D: 0.7607 Loss_G: 1.2838 D(x): 0.6232 D(G(z)): 0.1809 / 0.3559
[27/30][539/938] Loss_D: 0.9372 Loss_G: 2.9234 D(x): 0.8950 D(G(z

In [ ]:
# Show the loss functions of both discriminator and generator.
plt.figure(figsize=(12, 8))
plt.plot(np.arange(opt.niter), g_losses, lw=2, color="b")
plt.grid(True)
plt.title("Loss function of Generator")

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(np.arange(opt.niter), d_losses, lw=2, color="b")
plt.grid(True)
plt.title("Loss function of Discriminator")

## 我们来看一张真实的MNIST图片：

![](./figs/real_samples.png)

## 接下来我们依次来看一下生成的图片，一共10张，分别是在训练的10个循环中获得的

![](./figs/fake_samples_epoch_000.png)
![](./figs/fake_samples_epoch_001.png)
![](./figs/fake_samples_epoch_002.png)
![](./figs/fake_samples_epoch_003.png)
![](./figs/fake_samples_epoch_004.png)
![](./figs/fake_samples_epoch_005.png)
![](./figs/fake_samples_epoch_006.png)
![](./figs/fake_samples_epoch_007.png)
![](./figs/fake_samples_epoch_008.png)
![](./figs/fake_samples_epoch_009.png)